<a href="https://colab.research.google.com/github/shibakyrc0123-arch/A-Gps-Versiones/blob/main/Ver_1_0_%7C_A_Gps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from datetime import timedelta
import time

# --- PASO 1: BUSCADOR AUTOMÁTICO DE TU ARCHIVO ---
def encontrar_archivo():
    # Buscamos en la carpeta raiz y en sample_data
    rutas_a_buscar = ['/content/', '/content/sample_data/']

    print("🔍 Buscando archivo Excel o CSV...")

    for ruta_base in rutas_a_buscar:
        if os.path.exists(ruta_base):
            archivos = os.listdir(ruta_base)
            for archivo in archivos:
                # Buscamos algo que termine en .xls, .xlsx o .csv y que NO sea los de ejemplo
                if (archivo.endswith('.xls') or archivo.endswith('.xlsx') or archivo.endswith('.csv')) \
                   and "california_housing" not in archivo and "mnist" not in archivo:
                    ruta_completa = os.path.join(ruta_base, archivo)
                    print(f"✅ Archivo encontrado: {ruta_completa}")
                    return ruta_completa

    print("❌ No encontré ningún archivo. Por favor sube el archivo de nuevo en el panel de la izquierda.")
    return None

# --- CONFIGURACIÓN DEL ALGORITMO ---
bodega_coords = [
    (4.101740989185771, -73.65614987790829),
    (4.10275779142336, -73.65779357282884),
    (4.103576674506101, -73.65722519691684),
    (4.103303969059117, -73.65525092470286)
]
bodega_polygon = Polygon(bodega_coords)
geolocator = Nominatim(user_agent="colab_tracker_final")

def obtener_direccion(lat, lon):
    try:
        time.sleep(1.1)
        location = geolocator.reverse((lat, lon), exactly_one=True)
        return location.address if location else "Ubicación desconocida"
    except:
        return "Error geoloc"

def procesar_logica_vehiculo(archivo):
    print(f"📂 Procesando: {archivo}")

    # INTENTO DE LECTURA ROBUSTO
    try:
        if archivo.endswith('.csv'):
            df = pd.read_csv(archivo)
        else:
            # Intentamos leer Excel. Si es un HTML disfrazado, fallará y avisará.
            try:
                df = pd.read_excel(archivo)
            except Exception as e:
                # A veces los GPS bajan archivos HTML con extensión .xls
                print(f"⚠️ Error leyendo Excel normal: {e}. Intentando como HTML...")
                lista_tablas = pd.read_html(archivo)
                df = lista_tablas[0] # Tomamos la primera tabla

    except Exception as e:
        return f"🔥 ERROR CRÍTICO LEYENDO EL ARCHIVO: {e}"

    # Limpieza
    if 'FHServer' in df.columns:
        df = df.drop(columns=['FHServer'])

    # Validar columnas
    if 'Evento' not in df.columns or 'Coords' not in df.columns:
        return f"❌ El archivo no tiene las columnas 'Evento' o 'Coords'. Tiene: {list(df.columns)}"

    df['FhEvento'] = pd.to_datetime(df['FhEvento'])
    df = df.sort_values(by='FhEvento').reset_index(drop=True)

    # Filtro Año
    if not df.empty:
        anio = df['FhEvento'].iloc[0].year
        df = df[df['FhEvento'].dt.year == anio]

    reporte_final = []
    primer_encendido = False

    estacionado_ini = None
    estacionado_fin = None
    estacionado_pos = None
    dentro_bodega = False

    print("⏳ Generando reporte (espere un momento)...")

    for i, row in df.iterrows():
        fecha = row['FhEvento']
        evento = str(row['Evento'])

        try:
            coords_clean = str(row['Coords']).replace(" ", "")
            lat, lon = map(float, coords_clean.split(','))
            punto = Point(lat, lon)
        except:
            continue

        # 1. BODEGA MONTE CARLO
        esta_dentro = bodega_polygon.contains(punto)
        hora_fmt = fecha.strftime('%I:%M %p').lower()

        if esta_dentro and not dentro_bodega:
            reporte_final.append(f"-{hora_fmt} Entrada Bodega Monte Carlo")
            dentro_bodega = True
        elif not esta_dentro and dentro_bodega:
            reporte_final.append(f"-{hora_fmt} Salida Bodega Monte Carlo")
            dentro_bodega = False

        # 2. ENCENDIDO
        if evento == "Vehículo encendido" and not primer_encendido:
            dir = obtener_direccion(lat, lon)
            reporte_final.append(f"-{hora_fmt} Se enciende en {dir}")
            primer_encendido = True

        # 3. ESTACIONADO
        es_estacionado = (evento == "Vehiculo Estacionado")
        if estacionado_ini and not es_estacionado:
             dist = geodesic(estacionado_pos, (lat, lon)).meters
             if dist < 50: es_estacionado = True

        if es_estacionado:
            if not estacionado_ini:
                estacionado_ini = fecha
                estacionado_pos = (lat, lon)
            estacionado_fin = fecha
        else:
            if estacionado_ini:
                duracion = (estacionado_fin - estacionado_ini).total_seconds() / 60
                if duracion > 2:
                    dir = obtener_direccion(*estacionado_pos)
                    h_ini = estacionado_ini.strftime('%I:%M')
                    h_fin = estacionado_fin.strftime('%I:%M %p').lower()
                    reporte_final.append(f"-{h_ini} / {h_fin} Vehículo estacionado en {dir}")
                estacionado_ini = None
                estacionado_fin = None

        # 4. ENTRADA/SALIDA CIUDAD
        if (evento.startswith("Entrada") or evento.startswith("Salida")) and "Bodega" not in evento:
             reporte_final.append(f"-{hora_fmt} {evento}")

        # 5. REPORTE POR TIEMPO
        if evento == "Reporte por tiempo":
             dir_mov = obtener_direccion(lat, lon)
             reporte_final.append(f"-{hora_fmt} En movimiento por {dir_mov}")

    return ", ".join(reporte_final) + "."

# --- EJECUCIÓN ---
archivo_encontrado = encontrar_archivo()

if archivo_encontrado:
    resultado = procesar_logica_vehiculo(archivo_encontrado)
    print("\n--- RESULTADO FINAL ---")
    print(resultado)
else:
    print("⚠️ No pude iniciar. Sube el archivo nuevamente.")

🔍 Buscando archivo Excel o CSV...
✅ Archivo encontrado: /content/sample_data/reporte-850AHG-09-02-2026-13-46-17.xlsx
📂 Procesando: /content/sample_data/reporte-850AHG-09-02-2026-13-46-17.xlsx

--- RESULTADO FINAL ---
❌ El archivo no tiene las columnas 'Evento' o 'Coords'. Tiene: ['EVENTO', 'FHEVENTO', 'FHSERVER', 'VEL', 'COORDS', 'ODOM']
